#### Import Necessari

In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsOneClassifier

#### Caricamento Dataset

In [3]:
train1 = pd.read_json('dataset/Dataset20-21.json');
train2 = pd.read_json('dataset/Dataset21-22.json');
trainSet = pd.concat([train1, train2], ignore_index=True)
validationSet = pd.read_json('dataset/Dataset22-23.json');
testSet = pd.read_json('dataset/Dataset23-24.json');
trainSet.loc[trainSet["risultato"] == "X", "risultato"] = 3
trainSet.loc[trainSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "x", "risultato"] = 3
validationSet.loc[validationSet["risultato"] == "X", "risultato"] = 3

In [4]:
combined_data = pd.concat([trainSet, validationSet], axis=0, ignore_index=True)

In [5]:
X_combined = combined_data.drop("risultato", axis=1)
y_combined = combined_data["risultato"]

X_test = testSet.drop("risultato", axis=1)
y_test = testSet["risultato"]

In [6]:
X_combined_noName = X_combined
X_combined_noName = X_combined_noName.drop("squadraa", axis=1)
X_combined_noName = X_combined_noName.drop("squadrab", axis=1)
X_combined_noName = X_combined_noName.astype(float)
y_combined=y_combined.astype(float)

X_test_noName = X_test
X_test_noName = X_test_noName.drop("squadraa", axis=1)
X_test_noName = X_test_noName.drop("squadrab", axis=1)
X_test_noName = X_test_noName.astype(float)
#y_combined=y_combined.astype(float)

#### Inizio Codice training

In [7]:
svm_classifier = SVC(kernel='rbf') # OvO per la classificazione multiclasse
svm_classifier = OneVsOneClassifier(svm_classifier)

In [8]:
mask = ~np.isnan(y_combined)
y_combined = y_combined[mask]
X_combined_noName = X_combined_noName[mask] #Rimozione righe con NaN Values
cross_val_scores = cross_val_score(svm_classifier, X_combined_noName, y_combined, cv=10, scoring='accuracy')

In [10]:
svm_classifier.fit(X_combined_noName, y_combined)
y_pred_test = svm_classifier.predict(X_test_noName)

print("Accuratezza media della cross-validation:", cross_val_scores.mean())
#classification_rep = classification_report(y_test, y_pred_test)
#print("Report di classificazione sui dati di test:\n", classification_rep)


Accuratezza media della cross-validation: 0.4986104642136315
